In [ ]:
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(
        host='34.28.167.157',  # Localhost for the proxy
        port=3306,         # Port where the proxy is listening
        user='aaruldhawan',
        password='enter_password',
        database='test_databoose'
    )
    if connection.is_connected():
        print('Connected to MySQL database')
except Error as e:
    print('Error while connecting to MySQL', e)
finally:
    if connection.is_connected():
        cursor = connection.cursor()
        country_create_query = """
        CREATE TABLE Country(
        CountryName VARCHAR(100) PRIMARY KEY,
        IncomeGroup VARCHAR(255),
        CountryCode VARCHAR(255),
        Region VARCHAR(255)
        );"""
        disaster_create_query = """
        CREATE TABLE NaturalDisaster (
        DisasterID INT PRIMARY KEY,
        CountryName VARCHAR(100),
        Type VARCHAR(100),
        Year INT,
        Intensity FLOAT,
        FOREIGN KEY (CountryName) REFERENCES Country(CountryName)
        );
        """
        macro_create_query = """
        CREATE TABLE MacroeconomicImpact (
        DisasterID INT,
        CountryName VARCHAR(100),
        GDP_Loss DECIMAL(10,2),
        GDP_Growth_Change DECIMAL(5,2),
        Inflation_Change DECIMAL(5,2),
        Unemployment_Change DECIMAL(5,2),
        PRIMARY KEY (DisasterID, CountryName),
        FOREIGN KEY (DisasterID) REFERENCES NaturalDisaster(DisasterID),
        FOREIGN KEY (CountryName) REFERENCES Country(CountryName)
        );
        """
        micro_create_query = """
        CREATE TABLE MicroeconomicImpact (
        DisasterID INT,
        CountryName VARCHAR(100),
        Agriculture_Growth DECIMAL(5,2),
        Industry_Growth DECIMAL(5,2),
        Manufacturing_Growth DECIMAL(5,2),
        Services_Growth DECIMAL(5,2),
        Exports_Change DECIMAL(5,2),
        Imports_Change DECIMAL(5,2),
        PRIMARY KEY (DisasterID, CountryName),
        FOREIGN KEY (DisasterID) REFERENCES NaturalDisaster(DisasterID),
        FOREIGN KEY (CountryName) REFERENCES Country(CountryName)
        );
        """
        user_create_query = """
        CREATE TABLE User (
        UserID INT PRIMARY KEY,
        Username VARCHAR(255),
        Email VARCHAR(255) UNIQUE,
        Role VARCHAR(50)
        );
        """
        direct_create_query = """
        CREATE TABLE DirectDamage (
        DamageID INT PRIMARY KEY,
        DisasterID INT,
        CountryName VARCHAR(100),
        PropertyDamage DECIMAL(12,2),
        HousesDestroyed INT,
        Injuries INT,
        Deaths INT,
        FOREIGN KEY (DisasterID) REFERENCES NaturalDisaster(DisasterID),
        FOREIGN KEY (CountryName) REFERENCES Country(CountryName)
        );
        """
        government_response_create_query = """
        CREATE TABLE GovernmentResponse (
        ResponseID INT PRIMARY KEY,
        DisasterID INT,
        CountryName VARCHAR(100),
        AidAmount DECIMAL(12,2),
        PolicyChanges TEXT,
        RecoveryMeasures TEXT,
        FOREIGN KEY (DisasterID) REFERENCES NaturalDisaster(DisasterID),
        FOREIGN KEY (CountryName) REFERENCES Country(CountryName)
        );"""

        cursor.execute(country_create_query)
        cursor.execute(disaster_create_query)
        print("disaster added")
       # cursor.execute(micro_create_query)
        #cursor.execute(macro_create_query)
        print("Micro/Macro added")
        cursor.execute(user_create_query)
        print("User added")
        #cursor.execute(direct_create_query)
        print("Direct Damage Added")
        #cursor.execute(government_response_create_query)
        print("response added")

        connection.close()
        print('MySQL connection closed')

In [ ]:
import pandas as pd
df_income = pd.read_csv("/Users/aaruldhawan/Desktop/UIUC Classes/CS 411/sp25-cs411-team016-JeremyRenner/datasets/agriculture_datsets/income_groups.csv")
df_income = df_income.drop(df_income.columns[-1], axis=1)
df_income = df_income.drop("SpecialNotes", axis=1)
df_income = df_income.dropna(subset=['IncomeGroup', 'Region']).reset_index().drop("index", axis=1)
df_income.to_csv("cleaned_income_groups.csv")
df_income

In [ ]:
import mysql.connector
#Adding Country Table 
try:
    connection = mysql.connector.connect(
        host='34.28.167.157',
        port=3306,
        user='aaruldhawan',
        password='scarjoe',
        database='test_databoose'
    )
    cursor = connection.cursor()
    
    for _, row in df_income.iterrows():
        country = row["TableName"]
        region = row["Region"]
        income = row["IncomeGroup"]
        code = row["Country Code"]
        tuple1 = (country, income, code, region)
        
        print(code)
        try:
            cursor.execute("""
                INSERT INTO Country (`CountryName`, IncomeGroup, CountryCode, `Region`)
                VALUES (%s, %s, %s, %s)
            """, tuple1)
        except mysql.connector.Error as e:
            print(f"Error inserting {code}: {e}")
    
    connection.commit()

except mysql.connector.Error as e:
    print(f"Database error: {e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()


In [ ]:
import pandas as pd
earthquake_df = pd.read_csv('/Users/aaruldhawan/Desktop/UIUC Classes/CS 411/sp25-cs411-team016-JeremyRenner/datasets/natural_disaster_datasets/earthquakes-event.tsv', sep='\t')
tsunami_df = pd.read_csv('/Users/aaruldhawan/Desktop/UIUC Classes/CS 411/sp25-cs411-team016-JeremyRenner/datasets/natural_disaster_datasets/tsunamis-event.tsv', sep = '\t')
volcano_df = pd.read_csv('/Users/aaruldhawan/Desktop/UIUC Classes/CS 411/sp25-cs411-team016-JeremyRenner/datasets/natural_disaster_datasets/volcano-events.tsv', sep = '\t')



In [ ]:
country_list = list(df_income["TableName"])
country_list
lower_countries = [c.lower() for c in country_list]

In [ ]:
from rapidfuzz import process
match ,_,_ = process.extractOne("MYANMAR (BURMA)".lower(), lower_countries)
country = country_list[lower_countries.index(match)]
country

In [ ]:
earthquake_df

In [ ]:
tsunami_df

In [ ]:
volcano_df

In [ ]:
volcano_df = volcano_df.dropna(subset=['Country', 'Year'])
disaster_df = []
for _ , r in volcano_df.iterrows():
    country, _ , _ = process.extractOne(r["Country"].lower(), lower_countries)
    country = country_list[lower_countries.index(country)]
    year = r["Year"]
    vol = r["VEI"]
    dis = {'DisasterID' : len(disaster_df), 'CountryName': country, "Year": year, "Intensity": vol, "Type": "Volcano"}
    disaster_df.append(dis)
disaster_df

In [ ]:
tsunami_df = tsunami_df.dropna(subset=['Country', 'Year'])
for _ , r in tsunami_df.iterrows():
    country, _ , _ = process.extractOne(r["Country"].lower(), lower_countries)
    country = country_list[lower_countries.index(country)]
    year = r["Year"]
    vol = r["Tsunami Intensity"]
    dis = {'DisasterID' : len(disaster_df), 'CountryName': country, "Year": year, "Intensity": vol, "Type": "Tsunami"}
    disaster_df.append(dis)
disaster_df


In [ ]:
print(earthquake_df.columns)
earthquake_df = earthquake_df.dropna(subset=['Location Name', 'Year'])
for _ , r in earthquake_df.iterrows():
    country = r["Location Name"].split(':')[0].split(';')[0]
    country, _ , _ = process.extractOne(country.lower(), lower_countries)
    country = country_list[lower_countries.index(country)]
    year = r["Year"]
    vol = r["Mag"]
    dis = {'DisasterID' : len(disaster_df), 'CountryName': country, "Year": year, "Intensity": vol, "Type": "Earthquake"}
    disaster_df.append(dis)
disaster_df

In [ ]:
disaster_df = pd.DataFrame(disaster_df)
disaster_df.to_csv("disaster.csv")

In [ ]:
disaster_df['Intensity'] = disaster_df['Intensity'].fillna(0)

In [ ]:
import mysql.connector
#Adding Country Table 
try:
    connection = mysql.connector.connect(
        host='34.28.167.157',
        port=3306,
        user='aaruldhawan',
        password='scarjoe',
        database='test_databoose'
    )
    cursor = connection.cursor()
    
    for _, row in disaster_df.iterrows():
        disID = row["DisasterID"]
        country = row["CountryName"]
        year = row["Year"]
        inte = row["Intensity"]
        type1 = row["Type"]
        tuple1 = (disID, country, type1, year, inte)
        
        print(disID)
        try:
            cursor.execute("""
                INSERT INTO NaturalDisaster (`DisasterID`, CountryName, Type, `Year`, Intensity)
                VALUES (%s, %s, %s, %s, %s)
            """, tuple1)
        except mysql.connector.Error as e:
            print(f"Error inserting {code}: {e}")
    
    connection.commit()

except mysql.connector.Error as e:
    print(f"Database error: {e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()


In [ ]:
import pandas as pd
volcano_df = volcano_df.dropna(subset=['Country', 'Year'])
damage_df = []
for _ , r in volcano_df.iterrows():
    country, _ , _ = process.extractOne(r["Country"].lower(), lower_countries)
    country = country_list[lower_countries.index(country)]
    damage_cost = r['Total Damage ($Mil)'] if pd.notna(r['Total Damage ($Mil)']) else 0
    damage_cost_scale = r['Total Damage Description'] if pd.notna(r['Total Damage Description']) else 0
    houses_destroyed = r['Houses Destroyed'] if pd.notna(r['Houses Destroyed']) else 0
    houses_destroyed_scale = r['Houses Destroyed Description'] if pd.notna(r['Houses Destroyed Description']) else 0
    injuries = r['Injuries'] if pd.notna(r['Injuries']) else 0
    deaths = r['Total Deaths'] if pd.notna(r['Total Deaths']) else 0
    dis = {'DamageID' : len(damage_df),'DisasterID' : len(damage_df), 'CountryName': country,"DamageScale": damage_cost,"DamageCostScale": damage_cost_scale, 'HousesDestroyed': houses_destroyed, "HousesDestroyedScale": houses_destroyed_scale, "Injuries": injuries, "Deaths": deaths}
    damage_df.append(dis)
damage_df

In [ ]:
tsunami_df = tsunami_df.dropna(subset=['Country', 'Year'])
for _ , r in tsunami_df.iterrows():
    country, _ , _ = process.extractOne(r["Country"].lower(), lower_countries)
    country = country_list[lower_countries.index(country)]
    damage_cost = r['Total Damage ($Mil)'] if pd.notna(r['Total Damage ($Mil)']) else 0
    damage_cost_scale = r['Total Damage Description'] if pd.notna(r['Total Damage Description']) else 0
    houses_destroyed = r['Houses Destroyed'] if pd.notna(r['Houses Destroyed']) else 0
    houses_destroyed_scale = r['Houses Destroyed Description'] if pd.notna(r['Houses Destroyed Description']) else 0
    injuries = r['Injuries'] if pd.notna(r['Injuries']) else 0
    deaths = r['Total Deaths'] if pd.notna(r['Total Deaths']) else 0
    dis = {'DamageID' : len(damage_df),'DisasterID' : len(damage_df), 'CountryName': country,"DamageScale": damage_cost,"DamageCostScale": damage_cost_scale, 'HousesDestroyed': houses_destroyed, "HousesDestroyedScale": houses_destroyed_scale, "Injuries": injuries, "Deaths": deaths}
    damage_df.append(dis)
damage_df


In [ ]:
earthquake_df = earthquake_df.dropna(subset=['Location Name', 'Year'])
for _ , r in earthquake_df.iterrows():
    country = r["Location Name"].split(':')[0].split(';')[0]
    country, _ , _ = process.extractOne(country.lower(), lower_countries)
    country = country_list[lower_countries.index(country)]
    damage_cost = r['Total Damage ($Mil)'] if pd.notna(r['Total Damage ($Mil)']) else 0
    damage_cost_scale = r['Total Damage Description'] if pd.notna(r['Total Damage Description']) else 0
    houses_destroyed = r['Houses Destroyed'] if pd.notna(r['Houses Destroyed']) else 0
    houses_destroyed_scale = r['Houses Destroyed Description'] if pd.notna(r['Houses Destroyed Description']) else 0
    injuries = r['Injuries'] if pd.notna(r['Injuries']) else 0
    deaths = r['Total Deaths'] if pd.notna(r['Total Deaths']) else 0
    dis = {'DamageID' : len(damage_df),'DisasterID' : len(damage_df), 'CountryName': country,"DamageScale": damage_cost,"DamageCostScale": damage_cost_scale, 'HousesDestroyed': houses_destroyed, "HousesDestroyedScale": houses_destroyed_scale, "Injuries": injuries, "Deaths": deaths}
    damage_df.append(dis)
damage_df

In [ ]:
damage_df = pd.DataFrame(damage_df)
damage_df

In [ ]:
import mysql.connector
#Adding Country Table 

direct_create_query = """
        CREATE TABLE DirectDamage (
        DamageID INT PRIMARY KEY,
        DisasterID INT,
        CountryName VARCHAR(100),
        TotalDamage INT,
        TotalDamageScale INT,
        HousesDestroyed INT,
        HousesDestroyedScale INT,
        Injuries INT,
        Deaths INT,
        FOREIGN KEY (DisasterID) REFERENCES NaturalDisaster(DisasterID),
        FOREIGN KEY (CountryName) REFERENCES Country(CountryName)
        );
        """
try:
    connection = mysql.connector.connect(
        host='34.28.167.157',
        port=3306,
        user='aaruldhawan',
        password='scarjoe',
        database='test_databoose'
    )
    cursor = connection.cursor()
    cursor.execute(direct_create_query)
    for _, row in damage_df.iterrows():
        damID = row["DamageID"]
        disID = row["DisasterID"]
        country = row["CountryName"]
        cost = row["DamageScale"]
        cost_scale = row["DamageCostScale"]
        house = row["HousesDestroyed"]
        house_scale = row["HousesDestroyedScale"]
        inj = row["Injuries"]
        death = row["Deaths"]
        tuple1 = (damID, disID, country, cost, cost_scale, house, house_scale, inj, death)
        
        print(disID)
        try:
            cursor.execute("""
                INSERT INTO DirectDamage (DamageID, DisasterID, CountryName, TotalDamage, TotalDamageScale, HousesDestroyed, HousesDestroyedScale, Injuries, Deaths)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, tuple1)
        except mysql.connector.Error as e:
            print(f"Error inserting {code}: {e}")
    
    connection.commit()

except mysql.connector.Error as e:
    print(f"Database error: {e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()

